In [29]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect()
%sql duckdb:///:memory:
%sql conn --alias duckdb

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [30]:
%%sql
CREATE TABLE CollectedResults (
   RACE                 TEXT
  ,YEAR                 INTEGER
  ,STAGE_ID             TEXT
  ,STAGE_DATE           DATE
  ,STAGE_NAME           TEXT
  ,STAGE_PROFILE        TEXT
  ,STAGE_TYPE           TEXT
  ,RANK                 TEXT
  ,RIDER                TEXT
  ,TEAM                 TEXT
  ,UCI_POINTS           DOUBLE
  ,PCS_POINTS           DOUBLE
  ,UPDATE_TIMESTAMP     DATETIME
);

INSERT INTO CollectedResults
SELECT 
   RaceName
  ,RaceYear
  ,StageNumber
  ,CASE 
    WHEN LEN(StageDate) = 10 THEN CAST(StageDate AS DATE)
    ELSE MAKE_DATE(
      RaceYear, 
      CAST(SUBSTRING(StageDate, 4, 2) AS INTEGER), 
      CAST(SUBSTRING(StageDate, 1, 2) AS INTEGER)
    )
   END
  ,StageName
  ,''
  ,StageType
  ,RiderRank
  ,TRIM(SUBSTRING(RiderName, 1, POSITION(TRIM(TeamName) IN RiderName) - 1))
  ,TeamName
  ,TRY_CAST(UCIPoints AS DOUBLE)
  ,TRY_CAST(PCSPoints AS DOUBLE)
  ,UpdateTimeStamp
FROM read_csv_auto("../data/results_races/*.csv", union_by_name = True);

UPDATE CollectedResults src
SET STAGE_DATE = (
  SELECT MAX(sub.STAGE_DATE)
  FROM CollectedResults sub
  WHERE sub.RACE = src.RACE AND sub.YEAR = src.YEAR
)
WHERE src.STAGE_NAME = 'GC' AND src.STAGE_DATE IS NULL;

COPY CollectedResults TO '../data/results/results_2025_full.csv' (HEADER, DELIMITER ',');

,Count
0,33564


In [31]:
%%sql
CREATE TABLE Riders             AS FROM "../data/riders.csv";
CREATE TABLE RidersSelectedBy   AS FROM "../data/riders_selected_by.csv";
CREATE TABLE RiderTeams         AS FROM "../data/rider_teams.csv";
CREATE TABLE Managers           AS FROM "../data/managers.csv";
CREATE TABLE ManagerTeams       AS FROM "../data/manager_teams.csv";
CREATE TABLE ManagerTeamsCheapo AS FROM "../data/manager_cheapo_teams.csv";
CREATE TABLE PointsSystem       AS FROM "../data/points_system.csv";
CREATE TABLE Races              AS FROM "../data/races.csv";
CREATE TABLE Results_2023       AS FROM "../data/results/results_2023_full.csv";
CREATE TABLE Results_2024       AS FROM "../data/results/results_2024_full.csv";
CREATE TABLE Results_2025       AS FROM "../data/results/results_2025_full.csv";
CREATE TABLE CheapoBans         AS FROM "../data/cheapo_bans.csv";

,Count
0,14


In [32]:
%%sql
CREATE OR REPLACE TABLE RaceResultsPoints AS
SELECT
     results.Race
    ,races.RaceName
    ,races.RaceCategory
    ,results.Stage_ID
    ,COALESCE(results.Stage_Date, races.RaceStart) AS Stage_Date
    ,results.Stage_Type
    ,CASE WHEN results.Stage_Name <> 'GC' AND (results.Stage_Type <> 'TTT' OR results.Stage_Type IS NULL) THEN 1 ELSE 0 END AS Løbsdage
    ,results.Rider AS RiderName_PCS
    ,riders.RiderName_Zweeler
    ,riders.RiderPrice
    ,riders.RiderTeam
    ,COALESCE(selected.ValgtAf, 0) AS SelectedBy
    ,CASE WHEN cbans.RiderName IS NULL AND riders.RiderPrice <= 2.5 THEN 'Ja' ELSE '' END AS Cheapo
    ,results.Rank
    ,points.RacePoints AS Points
    ,CASE WHEN races.RaceName = 'Tour Down Under' THEN NULL Else points.RacePoints END AS CheapoPoints
FROM Riders riders
LEFT JOIN Results_2025 results ON UPPER(results.Rider) = UPPER(riders.RiderName_PCS)
LEFT JOIN Races races ON races.RaceName_PCS = results.Race
LEFT JOIN CheapoBans cbans ON cbans.RiderName = riders.RiderName_Zweeler
LEFT JOIN RidersSelectedBy selected ON selected.RiderName_Zweeler = riders.RiderName_Zweeler
LEFT JOIN PointsSystem points ON 1 = 1
    AND (results.Stage_Type IS NULL OR results.Stage_Type <> 'TTT')
    AND points.RaceRank = TRY_CAST(results.Rank AS INTEGER) 
    AND CASE WHEN results.Stage_Name LIKE '%Stage%' OR results.Stage_Name LIKE '%Prologue%' THEN REPLACE(races.RaceCategory, 'Tour', 'Stage') ELSE races.RaceCategory END = points.RaceCategory;

,Count
0,30983


In [33]:
%%sql
rider_output << 
WITH
ManagerLists AS
(
    SELECT RiderName, STRING_AGG(ManagerName, ', ' ORDER BY ManagerName ASC) AS Managers
    FROM ManagerTeams
    GROUP BY RiderName
),
RiderOverview AS 
(
    SELECT
         RiderName_Zweeler AS Navn
        ,MAX(RiderTeam) AS Hold
        ,MAX(RiderPrice) AS Pris
        ,MAX(SelectedBy) AS Valgt
        ,CAST(SUM(Løbsdage) AS int) AS Løbsdage
        ,COALESCE(CAST(SUM(Points) AS int), 0) AS Point
    FROM RaceResultsPoints
    GROUP BY RiderName_Zweeler
)
SELECT 
     *
    ,ROUND(Point / Pris, 1) AS "P/mil"
    ,ROUND(CASE WHEN Løbsdage = 0 THEN 0 ELSE Point / Løbsdage END, 1) AS "P/dage"
    ,COALESCE((SELECT Managers FROM ManagerLists sub WHERE sub.RiderName = src.Navn), '') AS Managers
FROM RiderOverview src;

In [34]:
%%sql
rider_output_cheapo << 
WITH
ManagerLists AS
(
    SELECT RiderName, STRING_AGG(ManagerName, ', ' ORDER BY ManagerName ASC) AS Managers
    FROM ManagerTeamsCheapo
    GROUP BY RiderName
),
RiderOverview AS 
(
    SELECT
         RiderName_Zweeler AS Navn
        ,MAX(RiderTeam) AS Hold
        ,MAX(RiderPrice) AS Pris
        ,MAX(SelectedBy) AS Valgt
        ,CAST(SUM(Løbsdage) AS int) AS Løbsdage
        ,COALESCE(CAST(SUM(CheapoPoints) AS int), 0) AS Point
        ,MAX(Cheapo) AS Cheapo
    FROM RaceResultsPoints
    GROUP BY RiderName_Zweeler
)
SELECT 
     *
    ,ROUND(Point / Pris, 1) AS "P/mil"
    ,ROUND(CASE WHEN Løbsdage = 0 THEN 0 ELSE Point / Løbsdage END, 1) AS "P/dage"
    ,COALESCE((SELECT Managers FROM ManagerLists sub WHERE sub.RiderName = src.Navn), '') AS Managers
FROM RiderOverview src
WHERE Cheapo = 'Ja';

In [35]:
%%sql
WITH Src AS
(
    SELECT YEARWEEK(results.Stage_Date) AS Uge, teams.ManagerName AS Manager, CAST(SUM(Points) AS int) AS Point
    FROM RaceResultsPoints results
    INNER JOIN ManagerTeams teams ON teams.RiderName = results.RiderName_Zweeler
    WHERE 1 = 1
        AND Uge IS NOT NULL
    GROUP BY YEARWEEK(results.Stage_Date), teams.ManagerName
)
PIVOT Src
ON Uge
USING COALESCE(SUM(Point)::int, 0)
GROUP BY Manager
ORDER BY Manager;

,Manager,202504,202505,202506,202507,202508,202509,202510,202511,202512,...,202527,202528,202529,202530,202531,202532,202533,202534,202535,202536
0,Chrelle,312.0,100.0,67.0,130.0,673.0,72.0,280.0,424.0,125.0,...,76.0,366.0,335.0,757.0,52.0,122.0,132.0,384.0,421.0,32.0
1,Hustlersen,484.0,26.0,6.0,188.0,437.0,0.0,161.0,227.0,127.0,...,73.0,305.0,352.0,790.0,100.0,62.0,86.0,353.0,299.0,14.0
2,Jappo,156.0,130.0,15.0,130.0,631.0,139.0,379.0,425.0,141.0,...,87.0,373.0,293.0,567.0,51.0,63.0,171.0,377.0,392.0,24.0
3,Jarma,8.0,105.0,29.0,38.0,557.0,179.0,207.0,284.0,205.0,...,169.0,446.0,328.0,640.0,41.0,140.0,192.0,497.0,323.0,6.0
4,Kenk,283.0,49.0,106.0,94.0,539.0,110.0,225.0,428.0,167.0,...,56.0,350.0,217.0,601.0,203.0,257.0,58.0,354.0,324.0,37.0
5,Knak,100.0,146.0,15.0,35.0,552.0,96.0,344.0,293.0,153.0,...,165.0,385.0,327.0,761.0,17.0,29.0,70.0,238.0,344.0,24.0
6,Matti,54.0,36.0,214.0,123.0,607.0,69.0,27.0,344.0,118.0,...,58.0,403.0,243.0,487.0,85.0,70.0,75.0,358.0,333.0,51.0
7,Okholm,183.0,68.0,49.0,22.0,648.0,109.0,162.0,481.0,101.0,...,101.0,514.0,486.0,1003.0,18.0,59.0,109.0,384.0,194.0,6.0
8,Tommy,189.0,94.0,27.0,88.0,414.0,138.0,219.0,269.0,160.0,...,63.0,339.0,347.0,570.0,0.0,155.0,72.0,295.0,237.0,36.0
9,Visti,62.0,106.0,77.0,132.0,437.0,110.0,193.0,378.0,192.0,...,85.0,404.0,338.0,729.0,20.0,97.0,166.0,453.0,453.0,21.0


In [36]:
%%sql
dagens_rapport_avanceret <<
WITH Src AS
(
     SELECT 
           CAST(results.Stage_Date AS date) AS Dato
          ,managers.ManagerName AS Manager
          ,CAST(SUM(results.Løbsdage) AS int) AS Løbsdage
          ,SUM(CASE WHEN results.Løbsdage = 1 THEN results.RiderPrice END) AS Millioner
          ,CAST(SUM(results.Points) AS int) AS Point
     FROM RaceResultsPoints results
     LEFT JOIN ManagerTeams managers ON managers.RiderName = results.RiderName_Zweeler
     GROUP BY
           CAST(results.Stage_Date AS DATE)
          ,managers.ManagerName
), ManagersDatoer AS
(
     SELECT 
           Manager
          ,Dato FROM
     (SELECT DISTINCT Manager FROM Src) m
     CROSS JOIN (SELECT DISTINCT Dato FROM Src) d
     WHERE m.Manager IS NOT NULL AND d.Dato IS NOT NULL
)
SELECT
      managersdatoer.Manager
     ,managersdatoer.Dato
     ,COALESCE(summer.Løbsdage, 0) AS Løbsdage
     ,COALESCE(summer.Millioner, 0) AS Millioner
     ,COALESCE(summer.Point, 0) AS Point
     ,ROUND(CASE WHEN COALESCE(summer.Løbsdage, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Løbsdage, 0) END, 2) AS "P/rytter"
     ,ROUND(CASE WHEN COALESCE(summer.Millioner, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Millioner, 0) END, 2) AS "P/mil"
     ,(
          SELECT COALESCE(STRING_AGG(RiderPoints, ', ' ORDER BY Points DESC), '')
          FROM
               (   SELECT
                         CONCAT(
                              SUBSTRING(r.RiderName_Zweeler, 1, position(', ' IN r.RiderName_Zweeler) - 1), 
                              ' (',
                              CAST(SUM(r.Points) AS int), 
                              ')'
                         ) AS RiderPoints,
                         SUM(r.Points) AS Points
                    FROM RaceResultsPoints r
                    INNER JOIN ManagerTeams m ON m.RiderName = r.RiderName_Zweeler
                    WHERE 1 = 1
                         AND CAST(r.Stage_Date AS date) = managersdatoer.Dato
                         AND m.ManagerName = managersdatoer.Manager
                         AND r.Points IS NOT NULL
                    GROUP BY r.RiderName_Zweeler
                    ORDER BY SUM(r.Points) DESC
                    LIMIT 3
               ) t
      ) AS "Top 3 ryttere"
FROM ManagersDatoer managersdatoer
LEFT JOIN Src summer ON managersdatoer.Manager = summer.Manager AND managersdatoer.Dato = summer.Dato
ORDER BY 
      managersdatoer.Dato DESC
     ,COALESCE(summer.Point, 0) DESC;

In [37]:
%%sql
ugens_rapport_avanceret <<
WITH Src AS
(
     SELECT 
           YEARWEEK(CAST(results.Stage_Date AS date)) AS Uge
          ,managers.ManagerName AS Manager
          ,CAST(SUM(results.Løbsdage) AS int) AS Løbsdage
          ,SUM(CASE WHEN results.Løbsdage = 1 THEN results.RiderPrice END) AS Millioner
          ,CAST(SUM(results.Points) AS int) AS Point
     FROM RaceResultsPoints results
     LEFT JOIN ManagerTeams managers ON managers.RiderName = results.RiderName_Zweeler
     GROUP BY
           YEARWEEK(CAST(results.Stage_Date AS date))
          ,managers.ManagerName
), ManagersDatoer AS
(
     SELECT 
           Manager
          ,Uge FROM
     (SELECT DISTINCT Manager FROM Src) m
     CROSS JOIN (SELECT DISTINCT Uge FROM Src) d
     WHERE m.Manager IS NOT NULL AND d.Uge IS NOT NULL
)
SELECT
      managersdatoer.Manager
     ,managersdatoer.Uge
     ,COALESCE(summer.Løbsdage, 0) AS Løbsdage
     ,COALESCE(summer.Millioner, 0) AS Millioner
     ,COALESCE(summer.Point, 0) AS Point
     ,ROUND(CASE WHEN COALESCE(summer.Løbsdage, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Løbsdage, 0) END, 2) AS "P/rytter"
     ,ROUND(CASE WHEN COALESCE(summer.Millioner, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Millioner, 0) END, 2) AS "P/mil"
     ,(
          SELECT COALESCE(STRING_AGG(RiderPoints, ', ' ORDER BY Points DESC), '')
          FROM
               (   SELECT
                         CONCAT(
                              SUBSTRING(r.RiderName_Zweeler, 1, position(', ' IN r.RiderName_Zweeler) - 1), 
                              ' (',
                              CAST(SUM(r.Points) AS int), 
                              ')'
                         ) AS RiderPoints,
                         SUM(r.Points) AS Points
                    FROM RaceResultsPoints r
                    INNER JOIN ManagerTeams m ON m.RiderName = r.RiderName_Zweeler
                    WHERE 1 = 1
                         AND YEARWEEK(CAST(r.Stage_Date AS date)) = managersdatoer.Uge
                         AND m.ManagerName = managersdatoer.Manager
                         AND r.Points IS NOT NULL
                    GROUP BY r.RiderName_Zweeler
                    ORDER BY SUM(r.Points) DESC
                    LIMIT 3
               ) t
      ) AS "Top 3 ryttere"
FROM ManagersDatoer managersdatoer
LEFT JOIN Src summer ON managersdatoer.Manager = summer.Manager AND managersdatoer.Uge = summer.Uge
ORDER BY 
      managersdatoer.Uge DESC
     ,COALESCE(summer.Point, 0) DESC;

In [38]:
rider_output.to_html("../outputtables/RytteroversigtTabel.html", table_id = "filterabletable", index = False)
rider_output_cheapo.to_html("../outputtables/RytteroversigtCheapoTabel.html", table_id = "filterabletable", index = False)
dagens_rapport_avanceret.to_html("../outputtables/DagensRapportAvanceret.html", table_id = "filterabletable", index = False)
ugens_rapport_avanceret.to_html("../outputtables/UgensRapportAvanceret.html", table_id = "filterabletable", index = False)

In [39]:
%%sql
WITH Src AS
(
    SELECT 
        managers.ManagerName AS Manager
        ,YEAR(results.Stage_Date) * 100 + MONTH(results.Stage_Date) AS ÅrMåned
        ,SUM(results.Løbsdage * results.RiderPrice) AS SamletPris
        ,SUM(results.Løbsdage) AS SamledeLøbsdage
        ,SUM(results.Points) AS SamledePoint
    FROM RaceResultsPoints results
    LEFT JOIN ManagerTeams managers ON managers.RiderName = results.RiderName_Zweeler
    WHERE 1 = 1
        AND managers.ManagerName IS NOT NULL
    GROUP BY
        managers.ManagerName
        ,YEAR(results.Stage_Date) * 100 + MONTH(results.Stage_Date)
), Managers AS
(
    SELECT DISTINCT Manager
    FROM Src
)
SELECT 
     m.Manager
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202501), 0), 2) AS "202501"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202502), 0), 2) AS "202502"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202503), 0), 2) AS "202503"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202504), 0), 2) AS "202504"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202505), 0), 2) AS "202505"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202506), 0), 2) AS "202506"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202507), 0), 2) AS "202507"
    ,ROUND(COALESCE((SELECT SUM(SamletPris) FROM Src s WHERE s.Manager = m.Manager), 0), 2) AS "Millioner i alt"
    ,CAST(ROUND(COALESCE((SELECT SUM(SamledePoint) FROM Src s WHERE s.Manager = m.Manager), 0), 2) AS int) AS "Point i alt"
    ,ROUND(COALESCE((SELECT SUM(SamledePoint) / SUM(SamletPris) FROM Src s WHERE s.Manager = m.Manager), 0), 2) AS "Total"
FROM Managers m;

,Manager,202501,202502,202503,202504,202505,202506,202507,Millioner i alt,Point i alt,Total
0,Visti,0.46,0.41,0.62,1.18,0.43,0.43,0.33,15787.3,7695,0.49
1,Chrelle,1.30,0.54,0.62,1.22,0.33,0.64,0.32,16601.7,8905,0.54
2,Tommy,0.70,0.41,0.57,1.18,0.34,0.48,0.26,15674.4,7054,0.45
3,Jarma,0.07,0.46,0.59,1.34,0.19,0.33,0.30,15706.7,7223,0.46
4,Knak,0.51,0.46,0.59,1.10,0.36,0.48,0.30,15677.6,7329,0.47
5,Hustlersen,1.40,0.51,0.61,1.07,0.25,0.40,0.33,14521.8,6935,0.48
6,Okholm,0.91,0.45,0.58,0.91,0.29,0.56,0.36,16344.1,7789,0.48
7,Kenk,1.45,0.46,0.65,0.99,0.38,0.45,0.32,15598.5,7821,0.50
8,Matti,0.76,0.50,0.50,0.86,0.36,0.41,0.23,16677.9,6887,0.41
9,Jappo,0.83,0.52,0.68,1.07,0.42,0.48,0.27,15700.4,8020,0.51
